La **régression** est une technique d'apprentissage supervisé utilisée en statistiques et en apprentissage automatique pour modéliser et analyser les relations entre une variable cible (dépendante) et une ou plusieurs variables indépendantes (ou prédicteurs). L’objectif principal de la régression est de prédire des valeurs continues pour la variable cible en se basant sur les valeurs des variables indépendantes.

**Importer l'ensemble de données**

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)
df=pd.read_csv("cleaned_data.csv",sep=',',on_bad_lines='skip', low_memory=False)

**Un aperçu des données dans notre DataFrame**

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122305 entries, 0 to 122304
Data columns (total 26 columns):
 #   Column                                                 Non-Null Count   Dtype  
---  ------                                                 --------------   -----  
 0   code                                                   122305 non-null  object 
 1   product_name                                           119274 non-null  object 
 2   created_datetime                                       122305 non-null  object 
 3   quantity                                               109982 non-null  object 
 4   brands                                                 117822 non-null  object 
 5   categories                                             122305 non-null  object 
 6   categories_en                                          122305 non-null  object 
 7   pnns_groups_1                                          122305 non-null  object 
 8   pnns_groups_2                     

**Suppresssion des variables de type objet**

In [3]:
df=df.drop(['energy-kj_100g','code','nutrition-score-fr_100g','product_name','created_datetime','quantity','brands','categories','categories_en','pnns_groups_1','main_category_en','ingredients_text','countries_en','nutriscore_grade','product_name_lower','brands_lower'], axis = 1)

**Encodage de la variable nutriscore_grade**

In [4]:
# Assure-toi d'utiliser le bon nom de colonne

try:
    df = pd.get_dummies(df, columns=['pnns_groups_2'], drop_first=True)
except:
    print(df.columns)

**Affichage des premières lignes**

In [5]:
df.head()
print(df.columns)

Index(['nutriscore_score', 'energy-kcal_100g', 'fat_100g',
       'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'fruits-vegetables-nuts-estimate-from-ingredients_100g',
       'pnns_groups_2_Appetizers',
       'pnns_groups_2_Artificially sweetened beverages',
       'pnns_groups_2_Biscuits and cakes', 'pnns_groups_2_Bread',
       'pnns_groups_2_Breakfast cereals', 'pnns_groups_2_Cereals',
       'pnns_groups_2_Cheese', 'pnns_groups_2_Chocolate products',
       'pnns_groups_2_Dairy desserts', 'pnns_groups_2_Dressings and sauces',
       'pnns_groups_2_Dried fruits', 'pnns_groups_2_Eggs',
       'pnns_groups_2_Fats', 'pnns_groups_2_Fish and seafood',
       'pnns_groups_2_Fruit juices', 'pnns_groups_2_Fruit nectars',
       'pnns_groups_2_Fruits', 'pnns_groups_2_Ice cream',
       'pnns_groups_2_Legumes', 'pnns_groups_2_Meat',
       'pnns_groups_2_Milk and yogurt', 'pnns_groups_2_Nuts',
       'pnns_groups_2_Offals', 'pnns_groups_2_One-dish m

**Préparer les données pour prédire la variable nutriscore**

In [6]:
X = df.drop("nutriscore_score", axis = 1)
y = df["nutriscore_score"]

**Sépartion des données d'entrainnement et les données de test**

In [7]:
from sklearn.model_selection import train_test_split
import random
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=12)

**Normalisation des données**

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = scaler.fit(X_train)
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

**Un arbre de régression de type CART**

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(
    learning_rate=0.1,  # Réduit le pas d'apprentissage
    n_estimators=150,    # Réduit le nombre d'estimateurs
    max_depth=3,         # Limite la profondeur des arbres
    random_state=14,
    max_features="sqrt",
    min_samples_leaf=5,
    min_samples_split=20,
    subsample=0.7
)
model.fit(X_train_scaled,y_train)
y_pred = model.predict(X_test_scaled)
y_train_pred = model.predict(X_train_scaled)

**Evaluation du modèle**

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))


print("MAE:", mean_absolute_error(y_train, y_train_pred))
print("MSE:", mean_squared_error(y_train, y_train_pred))
print("R²:", r2_score(y_train, y_train_pred))

MAE: 1.2946700207353308
MSE: 3.6608604181604045
R²: 0.9529497789014976
MAE: 1.2742414282586765
MSE: 3.4533628636746028
R²: 0.9551103635274941


**Enregistrement du modèle**

In [11]:
import pickle

# Ouvrir un fichier en mode écriture binaire pour enregistrer le modèle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
